# Setting

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import os
os.chdir('/content/drive/MyDrive/コンペ/space titanic')

In [25]:
import pandas as pd
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [26]:
df_train.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [27]:
df_test.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0013_01      Earth      True  G/3/S  TRAPPIST-1e  27.0  False   
1     0018_01      Earth     False  F/4/S  TRAPPIST-1e  19.0  False   
2     0019_01     Europa      True  C/0/S  55 Cancri e  31.0  False   
3     0021_01     Europa     False  C/1/S  TRAPPIST-1e  38.0  False   
4     0023_01      Earth     False  F/5/S  TRAPPIST-1e  20.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck              Name  
0          0.0        0.0           0.0     0.0     0.0   Nelly Carsoning  
1          0.0        9.0           0.0  2823.0     0.0    Lerome Peckers  
2          0.0        0.0           0.0     0.0     0.0   Sabih Unhearfus  
3          0.0     6652.0           0.0   181.0   585.0  Meratz Caltilter  
4         10.0        0.0         635.0     0.0     0.0   Brence Harperez

# 前処理

In [28]:
#Enumerate Homeplanet
train = df_train.copy()

title_mapping = {"Europa": 1, "Earth": 2, "Mars": 3}
train['HomePlanet'] = train['HomePlanet'].map(title_mapping)
train['HomePlanet'] = train['HomePlanet'].fillna(0)

train[['Cabin_1', 'Cabin_2', 'Cabin_3']] = train['Cabin'].str.split('/', expand=True)
train.drop('Cabin', axis=1, inplace=True)

cabin_1_mapping = {"B": 1, "F": 2, "A": 3, "G": 4, "E": 5, "D": 6, "C": 7, "T": 8}
train['Cabin_1'] = train['Cabin_1'].map(cabin_1_mapping)
train['Cabin_1'] = train['Cabin_1'].fillna(0)

cabin_3_mapping = {"P": 1, "S": 2}
train['Cabin_3'] = train['Cabin_3'].map(cabin_3_mapping)
train['Cabin_3'] = train['Cabin_3'].fillna(0)

train['Cabin_2'] = train['Cabin_2'].fillna(9999)
train['Cabin_2'] = train['Cabin_2'].astype(int) + 1
train['Cabin_2'] = train['Cabin_2'].replace(10000, 0)


train['Fare'] = train[['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
cols = ['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
for col in cols:
    train[col] = train[col].fillna(train[col].median())


train.loc[train["Transported"] == 1, "Age"] = train[train["Transported"] == 1]["Age"].fillna(train[train["Transported"] == 1]["Age"].mean())
train.loc[train["Transported"] == 0, "Age"] = train[train["Transported"] == 0]["Age"].fillna(train[train["Transported"] == 0]["Age"].mean())

print(train["Age"].mean())

train.drop('Destination', axis=1, inplace=True)

vip_mapping = {False: 1, True: 2}
train['VIP'] = train['VIP'].map(vip_mapping)
train['VIP'] = train['VIP'].fillna(0)

cryo_mapping = {False: 1, True: 2}
train['CryoSleep'] = train['CryoSleep'].map(cryo_mapping)
train['CryoSleep'] = train['CryoSleep'].fillna(0)


train.drop(['Name', 'Transported', 'PassengerId'], axis = 1, inplace = True)

28.827968418500845


In [30]:
#Cleaning Tes
test = df_test.copy()
title_mapping = {"Europa": 1, "Earth": 2, "Mars": 3}
test['HomePlanet'] = test['HomePlanet'].map(title_mapping)
test['HomePlanet'] = test['HomePlanet'].fillna(0)

test[['Cabin_1', 'Cabin_2', 'Cabin_3']] = test['Cabin'].str.split('/', expand=True)
test.drop('Cabin', axis=1, inplace=True)

cabin_1_mapping = {"B": 1, "F": 2, "A": 3, "G": 4, "E": 5, "D": 6, "C": 7, "T": 8}
test['Cabin_1'] = test['Cabin_1'].map(cabin_1_mapping)
test['Cabin_1'] = test['Cabin_1'].fillna(0)

cabin_3_mapping = {"P": 1, "S": 2}
test['Cabin_3'] = test['Cabin_3'].map(cabin_3_mapping)
test['Cabin_3'] = test['Cabin_3'].fillna(0)

test['Cabin_2'] = test['Cabin_2'].fillna(9999)
test['Cabin_2'] = test['Cabin_2'].astype(int) + 1
test['Cabin_2'] = test['Cabin_2'].replace(10000, 0)


test['Fare'] = test[['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
cols = ['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
for col in cols:
    test[col] = test[col].fillna(test[col].median())

test['Age'] = test['Age'].fillna(28.827968418500845)

test.drop('Destination', axis=1, inplace=True)

vip_mapping = {False: 1, True: 2}
test['VIP'] = test['VIP'].map(vip_mapping)
test['VIP'] = test['VIP'].fillna(0)

cryo_mapping = {False: 1, True: 2}
test['CryoSleep'] = test['CryoSleep'].map(cryo_mapping)
test['CryoSleep'] = test['CryoSleep'].fillna(0)




test.drop(['Name', 'PassengerId'], axis = 1, inplace = True)

In [13]:
data = df_train.copy()

# Step 1: Check for missing values
missing_values = data.isnull().sum()

# Step 2: Data type conversion and initial transformations
# Convert CryoSleep and VIP to boolean
data['CryoSleep'] = data['CryoSleep'].map({'True': True, 'False': False})
data['VIP'] = data['VIP'].map({'True': True, 'False': False})

# Step 3: Feature engineering
# Split Cabin into deck, num, and side
data[['Deck', 'Num', 'Side']] = data['Cabin'].str.split('/', expand=True)

# Split PassengerId into group and number
data[['Group', 'GroupNum']] = data['PassengerId'].str.split('_', expand=True)

# Create a new feature: Total expenditure
data['TotalExpenditure'] = data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)

# Step 4: Check target distribution
target_distribution = data['Transported'].value_counts(normalize=True)

# Display the processed dataset overview and results
processed_overview = {
    "Missing Values": missing_values,
    "Target Distribution": target_distribution,
    "Columns After Feature Engineering": data.columns.tolist()
}

In [31]:
# trainとtestを保存する
train.to_csv('data/train_v2.csv', index=False)
test.to_csv('data/test_v2.csv', index=False)